# Preambolo
In questo notebook cerco di valutare le performance ed estrarre l'importanza delle features da alberi di decisione indotti nel seguente modo: suddivido il dataset in 4 fold, su 3 di queste fold eseguo una gridsearch per trovare gli hypeparameters ottimali rispetto ad una score (accuratezza o f1_score), induco un albero di decisione con gli hyperparamether appena trovati allenandolo sulle 3 fold sopracitate e valuto la sua performance sulla quarta fold inizialmente lasciata "da parte". Facendo ciò per tutte le possibili scelte di 3 fold su 4, otterrò 4 alberi di decisione distinti.
Ciò è condotto per la classificazione 1 e 6 vs All, 3 vs All, 4 vs All. Inoltre voglio valutare l'eventuale impatto che l'oversampling con SMOTE(NC) può avere su performance e importantanza delle feature. 

## Moduli importati

In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score

## Importo dataset

In [2]:
df = pd.read_excel('/home/sam/tirocinio/DB PAROTIDE DEF_corretto.xlsx', sheet_name='Sheet7')

## Samples e labels

In [3]:
df = df.loc[df['ESCLUDERE'] == 0]
df = df.loc[:, ['Segni macro malignità', 'ADC', 'TIC type', 'T2', 'COD ISTO']]
df['ADC'] = df['ADC'].apply(lambda s: str(s).replace(',', '.')).astype(float)

#samples per cui la feature TIC type non è ancora stata sottoposta a one hot encoding
#da utilizzare quando si fa augmentation con SMOTENC
X_not_encoded = df.iloc[:,:-1].copy()

#samples sottoposti a one hot encoding
#da utilizzare normalmente e quando si fa augmentation con SMOTE
X_encoded = pd.get_dummies(X_not_encoded, columns = ['TIC type'])

y = df['COD ISTO'].copy()
y.value_counts()

3    53
4    27
1    12
5     6
2     5
6     4
Name: COD ISTO, dtype: int64

## Nota 1
Nel dataset, tra i vari campi, ve ne è uno chamato "Escludere". I seguenti esperimenti, a meno che non sia diversamente specificato, non considerano i punti per i quali la feature "Escludere" è settata ad 1.
Inoltre, nei seguenti esperimenti, a meno che non sia diversamente specificato, utilizzo solo le seguenti features per l'induzione degli alberi: Segni macro malignità, ADC, TIC type, T2.
La label è sempre COD ISTO.
Per ora non considero l'età, o altri valori presenti nel DS.
Le labels che considererò nei seguenti esperimenti sono:
1 e 6 vs All, 3 vs All, 4 vs All

## Nota 2
La sottostante funzione permette di condurre l'esperimento descritto anche applicando augmentation con SMOTE o SMOTENC così da ridurre lo sbilanciamento tra classi. Nei seguenti esperimenti l'oversampling è applicato sul training set succesivamente alla creazione delle label di interesse. Ciò non determina alcuna differenza rispetto a fare oversampling PRIMA della creazione delle label nel caso della classificazione 3 vs All e 4 vs All. Tuttavia vi è una differenza nel caso 1 e 6 vs All, perchè successivamente alla creazione delle label i punti di ambo queste classi vengono mappati alla classe 1 e dunque SMOTE tratterà tali punti come se effettivamente appartenessero alla stessa classe. Non so come ciò possa impattare sull'oversampling dato il procedimento di interpolazione utilizzato da SMOTE.


## Funzione per condurre l'esperimento

In [4]:
#l'eventuale augmentation è fatta DOPO la creazione delle labels
#e ad ogni iterazione augmento di nuovo

def foo(X, y, labels, tuned_parameters, scoring_func = accuracy_score,
                 n_splits = 4, n_splits_gs = 3, random_state_tree = None,
                 n_it = 1, aug = None, aug_sampling_strategy = 'auto', k_neighbors = 5):
    
    
    #creazione scorer per GridSearch
    if scoring_func is f1_score:
        scoring = make_scorer(scoring_func, average = 'weighted')
        
    else: 
        scoring = make_scorer(scoring_func)
        
    
    #di default faccio una sola iterazione
    for i in range(n_it):
        print('### ITERATION %d ###\n' %(i+1))
        
        #di default creo 4 fold e prima di crearle mischio i punti
        skf = StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = None)
        
        for train_index, test_index in skf.split(X,y):
            
            #divido training set e test set
            #stratificato rispetto a tutte le classi del ds iniziale
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            #creazione label
            y_train = pd.Series([1 if x in labels else 0 for x in y_train])
            y_test = pd.Series([1 if x in labels else 0 for x in y_test])
            

            #eventuale data augmentation
            #di default porta il numero di samples della classe minoritaria al numero di quelli
            #della classe maggioritaria
            if aug == 'SMOTE':
                #se uso smote passo il dataset sui cui Tic type è stato reso
                #numerico con one hot encoding e tutte le feature sono considerate numeriche
                sm = SMOTE(sampling_strategy = aug_sampling_strategy,random_state = None,
                           k_neighbors = k_neighbors)
                X_train, y_train = sm.fit_resample(X_train, y_train)
                
            elif aug == 'SMOTENC':
                #se uso smote considero tutte le feature tranne ADC come categoriche
                #e passo i samples senza one hot encoding sulla feature Tic type
                sm = SMOTENC(sampling_strategy = aug_sampling_strategy,random_state = None,
                             k_neighbors = k_neighbors,categorical_features = [0,2,3])
                X_train, y_train = sm.fit_resample(X_train, y_train)
                
                #però ora devo ricordarmi di fare one hot encoding perchè i DT prendono solo
                #feature numeriche
                X_train = pd.get_dummies(X_train, columns = ['TIC type'])
                X_test = pd.get_dummies(X_test, columns = ['TIC type'])
            
            
            print('# Tuning hyper-parameters #\n')
            albero = tree.DecisionTreeClassifier(random_state = random_state_tree)
            
            #creazione oggetto che definisce le fold nella GridSearch
            cv = StratifiedKFold(n_splits = n_splits_gs, shuffle = True, random_state = None)
            
            #gridsearch
            clf = GridSearchCV(albero, param_grid = tuned_parameters, cv = cv, scoring = scoring,
                              refit = True)
            clf.fit(X_train,y_train)
            
            print('Parameters found: ')
            print(clf.best_params_)
            print('\nAverage cross-validated score of best_estimator_: %0.3f' %(clf.best_score_))
            
            #score valutata sul test set(la quarta fold esclusa dal training set)
            y_pred = clf.best_estimator_.predict(X_test)
            
            if scoring_func is f1_score:
                current_score = scoring_func(y_test, y_pred, average = 'weighted')
            else:
                current_score = scoring_func(y_test, y_pred)
                
            print('Score computed on test set: %0.3f' %(current_score))
                
            print('Feature importances of best_estimator_:\n')
            print(pd.Series(clf.best_estimator_.feature_importances_, index = X_train.columns))
            print('-<>-'*25)
            print()

## Seed per la riproducibilità e parametri da ottimizzare

In [5]:
seed = 42
tuned_parameters = {'criterion':['gini', 'entropy'],
                    'max_depth':list(range(1,6)),
                    'class_weight':[None, 'balanced']}

# Esperimenti

### 3 vs All, accuracy, no augmentation

In [6]:
foo(X_encoded, y, [3], tuned_parameters, random_state_tree = seed)

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.875
Score computed on test set: 0.889
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.865371
T2                       0.000000
TIC type_A               0.029641
TIC type_B               0.104988
TIC type_C               0.000000
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.925
Score computed on test set: 0.852
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.818177
T2                       0.000000
TIC type_A               0.157214
TIC type_B               0.000000
TIC type

### 3 vs All, f1_score, no augmentation

In [7]:
foo(X_encoded, y, [3],tuned_parameters, random_state_tree = seed, scoring_func = f1_score)

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.899
Score computed on test set: 0.887
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.868173
T2                       0.000000
TIC type_A               0.029409
TIC type_B               0.102417
TIC type_C               0.000000
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.862
Score computed on test set: 0.926
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.787712
T2                       0.000000
TIC type_A               0.212288
TIC type_B               0.000

### 3 vs All, accuracy, SMOTE augmentation (auto)

In [8]:
foo(X_encoded, y, [3], tuned_parameters, random_state_tree = seed, aug = 'SMOTE')

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.830
Score computed on test set: 0.926
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.818917
T2                       0.000000
TIC type_A               0.181083
TIC type_B               0.000000
TIC type_C               0.000000
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.875
Score computed on test set: 0.889
Feature importances of best_estimator_:

Segni macro malignità    0.00000
ADC                      0.86522
T2                       0.00000
TIC type_A               0.13478
TIC type_B               0.00000
TIC typ

### 3 vs All, accuracy, SMOTENC augmentation (auto)

In [9]:
foo(X_not_encoded, y, [3],tuned_parameters, random_state_tree = seed, aug = 'SMOTENC')

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.890
Score computed on test set: 0.889
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.854837
T2                       0.000000
TIC type_A               0.145163
TIC type_B               0.000000
TIC type_C               0.000000
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.887
Score computed on test set: 0.852
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.839522
T2                       0.000000
TIC type_A               0.160478
TIC type_B               0.000000
TI

### 3 vs All, f1, SMOTE augmentation (auto)

In [10]:
foo(X_encoded ,y, [3],tuned_parameters, random_state_tree = seed, aug = 'SMOTE', scoring_func = f1_score)

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'entropy', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.878
Score computed on test set: 0.889
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.809739
T2                       0.000000
TIC type_A               0.190261
TIC type_B               0.000000
TIC type_C               0.000000
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.911
Score computed on test set: 0.851
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.841115
T2                       0.000000
TIC type_A               0.062344
TIC type_B               0.096541
TIC t

### 3 vs All, f1, SMOTENC augmentation (auto)

In [11]:
foo(X_not_encoded, y, [3], tuned_parameters, random_state_tree = seed,
             aug = 'SMOTENC', scoring_func = f1_score)

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.879
Score computed on test set: 0.925
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.202234
T2                       0.000000
TIC type_A               0.797766
TIC type_B               0.000000
TIC type_C               0.000000
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.874
Score computed on test set: 0.889
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.868918
T2                       0.009562
TIC type_A               0.121521
TIC type_B               0.000000
TIC type

### 4 vs All, accuracy, no augmentation

In [12]:
foo(X_encoded, y, [4],tuned_parameters, random_state_tree = seed)

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 1.000
Score computed on test set: 1.000
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.117907
T2                       0.000000
TIC type_A               0.000000
TIC type_B               0.882093
TIC type_C               0.000000
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 1.000
Score computed on test set: 1.000
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.121212
T2                       0.000000
TIC type_A               0.000000
TIC type_B               0.878788
TIC type

### 4 vs All, f1, no augmentation

In [13]:
foo(X_encoded, y, [4],tuned_parameters, random_state_tree = seed, scoring_func = f1_score)

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 1}

Average cross-validated score of best_estimator_: 0.988
Score computed on test set: 0.929
Feature importances of best_estimator_:

Segni macro malignità    0.0
ADC                      0.0
T2                       0.0
TIC type_A               0.0
TIC type_B               1.0
TIC type_C               0.0
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 1.000
Score computed on test set: 1.000
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.173913
T2                       0.000000
TIC type_A               0.000000
TIC type_B               0.826087
TIC type_C               0.000000
dtyp

### 4 vs All, accuracy, SMOTE augmentation (auto)

In [14]:
foo(X_encoded, y, [4],tuned_parameters, random_state_tree = seed, aug = 'SMOTE')

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 1}

Average cross-validated score of best_estimator_: 0.983
Score computed on test set: 0.963
Feature importances of best_estimator_:

Segni macro malignità    0.0
ADC                      0.0
T2                       0.0
TIC type_A               0.0
TIC type_B               1.0
TIC type_C               0.0
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 1.000
Score computed on test set: 1.000
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.064516
T2                       0.000000
TIC type_A               0.000000
TIC type_B               0.935484
TIC type_C               0.000000
dtyp

### 4 vs All, accuracy, SMOTENC augmentation (auto)

In [15]:
foo(X_not_encoded, y, [4],tuned_parameters, random_state_tree = seed, aug = 'SMOTENC')

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 1}

Average cross-validated score of best_estimator_: 0.983
Score computed on test set: 0.963
Feature importances of best_estimator_:

Segni macro malignità    0.0
ADC                      0.0
T2                       0.0
TIC type_A               0.0
TIC type_B               1.0
TIC type_C               0.0
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 1.000
Score computed on test set: 1.000
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.064516
T2                       0.000000
TIC type_A               0.000000
TIC type_B               0.935484
TIC type_C               0.000000
dtyp

### 4 vs All, f1, SMOTE augmentation (auto)

In [16]:
foo(X_encoded,y, [4],tuned_parameters, random_state_tree = seed, aug = 'SMOTE', scoring_func = f1_score)

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 1}

Average cross-validated score of best_estimator_: 0.991
Score computed on test set: 0.929
Feature importances of best_estimator_:

Segni macro malignità    0.0
ADC                      0.0
T2                       0.0
TIC type_A               0.0
TIC type_B               1.0
TIC type_C               0.0
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 1.000
Score computed on test set: 1.000
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.095238
T2                       0.000000
TIC type_A               0.000000
TIC type_B               0.904762
TIC type_C               0.000000
dtyp

### 4 vs All, f1, SMOTENC  augmentation (auto)

In [17]:
foo(X_not_encoded, y, [4],tuned_parameters, random_state_tree = seed, aug = 'SMOTENC', scoring_func = f1_score)

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 1}

Average cross-validated score of best_estimator_: 0.983
Score computed on test set: 0.964
Feature importances of best_estimator_:

Segni macro malignità    0.0
ADC                      0.0
T2                       0.0
TIC type_A               0.0
TIC type_B               1.0
TIC type_C               0.0
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 1.000
Score computed on test set: 1.000
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.095238
T2                       0.000000
TIC type_A               0.000000
TIC type_B               0.904762
TIC type_C               0.000000
dtyp

### 1 e 6 vs All, accuracy, no augmentation

In [18]:
foo(X_encoded,y, [1,6],tuned_parameters, random_state_tree = seed)

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 4}

Average cross-validated score of best_estimator_: 0.962
Score computed on test set: 0.852
Feature importances of best_estimator_:

Segni macro malignità    0.451563
ADC                      0.154080
T2                       0.156250
TIC type_A               0.000000
TIC type_B               0.111607
TIC type_C               0.126500
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 1}

Average cross-validated score of best_estimator_: 0.938
Score computed on test set: 0.889
Feature importances of best_estimator_:

Segni macro malignità    1.0
ADC                      0.0
T2                       0.0
TIC type_A               0.0
TIC type_B               0.0
TIC type_C               0.0
dtyp

### 1 e 6 vs All, f1, no augmentation

In [19]:
foo(X_encoded,y, [1,6],tuned_parameters, random_state_tree = seed, scoring_func = f1_score)

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 1}

Average cross-validated score of best_estimator_: 0.926
Score computed on test set: 0.859
Feature importances of best_estimator_:

Segni macro malignità    1.0
ADC                      0.0
T2                       0.0
TIC type_A               0.0
TIC type_B               0.0
TIC type_C               0.0
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5}

Average cross-validated score of best_estimator_: 0.912
Score computed on test set: 0.926
Feature importances of best_estimator_:

Segni macro malignità    0.523630
ADC                      0.197984
T2                       0.140729
TIC type_A               0.000000
TIC type_B               0.050279
TIC type_C               0.087378
dtyp

### 1 e 6 vs All, accuracy, SMOTE augmentation (auto)

In [20]:
foo(X_encoded, y, [1,6],tuned_parameters, random_state_tree = seed, aug = 'SMOTE')

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 4}

Average cross-validated score of best_estimator_: 0.948
Score computed on test set: 0.741
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.444175
T2                       0.029258
TIC type_A               0.000000
TIC type_B               0.065273
TIC type_C               0.461293
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'entropy', 'max_depth': 5}

Average cross-validated score of best_estimator_: 0.912
Score computed on test set: 0.926
Feature importances of best_estimator_:

Segni macro malignità    0.025910
ADC                      0.392081
T2                       0.120164
TIC type_A               0.026394
TIC type_B               0.054915
TIC t

### 1 e 6 vs All, accuracy, SMOTENC augmentation (auto)

In [21]:
foo(X_not_encoded, y, [1,6],tuned_parameters, random_state_tree = seed, aug = 'SMOTENC')

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5}

Average cross-validated score of best_estimator_: 0.963
Score computed on test set: 0.926
Feature importances of best_estimator_:

Segni macro malignità    0.736842
ADC                      0.068350
T2                       0.099561
TIC type_A               0.008114
TIC type_B               0.000000
TIC type_C               0.087134
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 4}

Average cross-validated score of best_estimator_: 0.956
Score computed on test set: 0.926
Feature importances of best_estimator_:

Segni macro malignità    0.025495
ADC                      0.107018
T2                       0.240332
TIC type_A               0.000000
TIC type_B               0.008593
TIC type

### 1 e 6 vs All, f1, SMOTE augmentation (auto)

In [22]:
foo(X_encoded, y, [1,6],tuned_parameters, random_state_tree = seed, aug = 'SMOTE', scoring_func = f1_score)

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 4}

Average cross-validated score of best_estimator_: 0.956
Score computed on test set: 0.863
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.394203
T2                       0.018335
TIC type_A               0.000000
TIC type_B               0.000000
TIC type_C               0.587462
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 4}

Average cross-validated score of best_estimator_: 0.912
Score computed on test set: 0.852
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.268391
T2                       0.000000
TIC type_A               0.025282
TIC type_B               0.087785
TIC type

### 1 e 6 vs All, f1, SMOTENC augmentation (auto)

In [23]:
foo(X_not_encoded, y, [1,6],tuned_parameters, random_state_tree = seed, aug = 'SMOTENC', scoring_func = f1_score)

### ITERATION 1 ###

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 3}

Average cross-validated score of best_estimator_: 0.956
Score computed on test set: 0.926
Feature importances of best_estimator_:

Segni macro malignità    0.000000
ADC                      0.132600
T2                       0.153378
TIC type_A               0.000000
TIC type_B               0.000000
TIC type_C               0.714022
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-parameters #

Parameters found: 
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5}

Average cross-validated score of best_estimator_: 0.971
Score computed on test set: 0.926
Feature importances of best_estimator_:

Segni macro malignità    0.646199
ADC                      0.134211
T2                       0.088158
TIC type_A               0.007895
TIC type_B               0.000

## Conduco nuovamente gli esperimenti per la classificazione 1 e 6 vs All, solo che in questo caso eseguo la data augmentation prima della creazione delle label, cioè faccio oversampling dei punti 1 e punti 6 separatamente
L'obbiettivo è determinare se realizzare l'augmentation in maniera differente da come fatto prima porta ad importanze delle feature diverse.
In questo caso l'augmentation sarà condotta triplicando il numero di punti 1 e triplicando il numero di punti 6.
Tuttavia bisogna ricordare che utilizzando questo approccio,a causa del ridotto numero di punti appartenenti alla classe 6, è necessario settare il parametro k_neighbors ad un valore molto basso. Ciò potrebbe influire negativamente sulla qualità dell'augmentation

In [24]:
# Modifico opportunamente la funzione

def foo_variant(X, y, labels, tuned_parameters, scoring_func = accuracy_score,
                 n_splits = 4, n_splits_gs = 3, random_state_tree = None,
                 n_it = 1, aug = None, aug_sampling_strategy = 'auto', k_neighbors = 5):
    
    
    #creazione scorer per GridSearch
    if scoring_func is f1_score:
        scoring = make_scorer(scoring_func, average = 'weighted')
        
    else: 
        scoring = make_scorer(scoring_func)
        
    
    #di default faccio una sola iterazione
    for i in range(n_it):
        print('### ITERATION %d ###\n' %(i+1))
        
        #di default creo 4 fold e prima di crearle mischio i punti
        skf = StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = None)
        
        for train_index, test_index in skf.split(X,y):
            
            #divido training set e test set
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            
            
            #eventuale data augmentation
            if aug == 'SMOTE':
                sm = SMOTE(sampling_strategy = aug_sampling_strategy,random_state = None,
                           k_neighbors = k_neighbors)
                X_train, y_train = sm.fit_resample(X_train, y_train)
                
            elif aug == 'SMOTENC':
                sm = SMOTENC(sampling_strategy = aug_sampling_strategy,random_state = None,
                             k_neighbors = k_neighbors, categorical_features = [0,2,3])
                X_train, y_train = sm.fit_resample(X_train, y_train)
                X_train = pd.get_dummies(X_train, columns = ['TIC type'])
                X_test = pd.get_dummies(X_test, columns = ['TIC type'])
                
                
            #creazione label    
            y_train = pd.Series([1 if x in labels else 0 for x in y_train])
            y_test = pd.Series([1 if x in labels else 0 for x in y_test])
            
            
            print('# Tuning hyper-paramethers #\n')
            albero = tree.DecisionTreeClassifier(random_state = random_state_tree)
            
            #creazione oggetto che definisce le fold nella GridSearch
            #forse potevo portarlo anche fuori
            cv = StratifiedKFold(n_splits = n_splits_gs, shuffle = True, random_state = None)
            
            clf = GridSearchCV(albero, param_grid = tuned_parameters, cv = cv, scoring = scoring,
                              refit = True)
            clf.fit(X_train,y_train)
            
            print('Parameters found: ')
            print(clf.best_params_)
            print('\nAverage cross-validated score of best_estimator_: %0.3f' %(clf.best_score_))
            
            #score valutata sul test set(la quarta fold esclusa dal training)
            y_pred = clf.best_estimator_.predict(X_test)
            
            if scoring_func is f1_score:
                current_score = scoring_func(y_test, y_pred, average = 'weighted')
            else:
                current_score = scoring_func(y_test, y_pred)
                
            print('Score computed on test set: %0.3f' %(current_score))
                
            print('Feature importances for best_estimator_:\n')
            print(pd.Series(clf.best_estimator_.feature_importances_, index = X_train.columns))
            print('-<>-'*25)
            print()
            
  

## Seed, parametri da ottimizzare e parametri per l'oversampling

In [25]:
seed = 42
tuned_parameters = {'criterion':['gini', 'entropy'],
                    'max_depth':list(range(1,6)),
                    'class_weight':[None, 'balanced']}

aug_sampling_strategy = {1:y.value_counts()[1]*3,
                        6:y.value_counts()[1]*3}

#sono obbligato dallo scarso numero di punti per la classe 6
k_neighbors = 2

### 1 e 6 vs All, accuracy, SMOTE augmentation

In [26]:
foo_variant(X_encoded, y, [1,6],tuned_parameters, random_state_tree = seed,
             aug = 'SMOTE', aug_sampling_strategy = aug_sampling_strategy, k_neighbors = k_neighbors)

### ITERATION 1 ###

# Tuning hyper-paramethers #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

Average cross-validated score of best_estimator_: 0.900
Score computed on test set: 0.667
Feature importances for best_estimator_:

Segni macro malignità    0.000000
ADC                      0.448752
T2                       0.000000
TIC type_A               0.000000
TIC type_B               0.551248
TIC type_C               0.000000
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-paramethers #

Parameters found: 
{'class_weight': None, 'criterion': 'entropy', 'max_depth': 5}

Average cross-validated score of best_estimator_: 0.929
Score computed on test set: 0.815
Feature importances for best_estimator_:

Segni macro malignità    0.177276
ADC                      0.247638
T2                       0.257110
TIC type_A               0.000000
TIC type_B               0.308155
T

### 1 e 6 vs All, accuracy, SMOTENC augmentation

In [27]:
foo_variant(X_not_encoded, y, [1,6],tuned_parameters, random_state_tree = seed,
             aug = 'SMOTENC', aug_sampling_strategy = aug_sampling_strategy, k_neighbors = k_neighbors)

### ITERATION 1 ###

# Tuning hyper-paramethers #

Parameters found: 
{'class_weight': None, 'criterion': 'entropy', 'max_depth': 3}

Average cross-validated score of best_estimator_: 0.878
Score computed on test set: 0.778
Feature importances for best_estimator_:

Segni macro malignità    0.000000
ADC                      0.356187
T2                       0.081351
TIC type_A               0.088756
TIC type_B               0.000000
TIC type_C               0.473706
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-paramethers #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 3}

Average cross-validated score of best_estimator_: 0.957
Score computed on test set: 0.815
Feature importances for best_estimator_:

Segni macro malignità    0.032124
ADC                      0.208148
T2                       0.394227
TIC type_A               0.000000
TIC type_B               0.365501
T

### 1 e 6 vs All, f1, SMOTE augmentation

In [28]:
foo_variant(X_encoded, y, [1,6],tuned_parameters, random_state_tree = seed,
             aug = 'SMOTE', aug_sampling_strategy = aug_sampling_strategy,
             k_neighbors = k_neighbors, scoring_func = f1_score)

### ITERATION 1 ###

# Tuning hyper-paramethers #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 4}

Average cross-validated score of best_estimator_: 0.942
Score computed on test set: 0.834
Feature importances for best_estimator_:

Segni macro malignità    0.000000
ADC                      0.197170
T2                       0.003539
TIC type_A               0.353026
TIC type_B               0.446265
TIC type_C               0.000000
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-paramethers #

Parameters found: 
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5}

Average cross-validated score of best_estimator_: 0.878
Score computed on test set: 0.869
Feature importances for best_estimator_:

Segni macro malignità    0.044201
ADC                      0.419276
T2                       0.006580
TIC type_A               0.003244
TIC type_B               0.46867

### 1 e 6 vs All, f1, SMOTENC augmentation

In [29]:
foo_variant(X_not_encoded, y, [1,6],tuned_parameters, random_state_tree = seed,
             aug = 'SMOTENC', aug_sampling_strategy = aug_sampling_strategy,
             k_neighbors = k_neighbors, scoring_func = f1_score)

### ITERATION 1 ###

# Tuning hyper-paramethers #

Parameters found: 
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5}

Average cross-validated score of best_estimator_: 0.862
Score computed on test set: 0.900
Feature importances for best_estimator_:

Segni macro malignità    0.000000
ADC                      0.528325
T2                       0.052733
TIC type_A               0.048277
TIC type_B               0.000000
TIC type_C               0.370665
dtype: float64
-<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>--<>-

# Tuning hyper-paramethers #

Parameters found: 
{'class_weight': None, 'criterion': 'entropy', 'max_depth': 5}

Average cross-validated score of best_estimator_: 0.913
Score computed on test set: 0.863
Feature importances for best_estimator_:

Segni macro malignità    0.000000
ADC                      0.323871
T2                       0.057666
TIC type_A               0.099225
TIC type_B               0.000000
T

# Conlusioni
Nella classificazione 3 vs All, la feature più importante sembra essere di gran lunga il valore ADC. Un ruolo decisamente minore è svolto dal fatto che la curva tic sia di tipo A o meno. La data augmentation a volte mostra lo stesso risultato mentre in altri casi porta allo scambio di importanza tra le feature sopracitate.

Nella classificazione 4 vs All, sia con e senza augmentation la feature decisamente più rilevante è il fatto che la curva tic sia di tipo B o meno, seguita dal valore ADC che è comunque molto meno importante.

Nella classificazione 1 e 6 vs All i risultati ottenuti senza augmentation sembrano indicare i segni macroscopici di malignità come feature più rilevante, mentre l'importanza delle altre feature sembra variare in maniera maggiore da esperimento a esperimento. In particolare, l'utilizzo dell'augmentation porta a risultati poco consistenti tra loro.